# Plotting Fire Data on a Bubble Map 

## Using Plotly for Python 

[Plotly](https://plot.ly/#/) has a [Python library](https://plot.ly/python/) that allows you to create interactive data visualizations online, such as line plots, scatter maps, heatmaps, and 3D network graphs. In this tutorial, we will create a bubble map that visualizes wildfire data in California (based on the FIRMS data we acquired). 

Follow the [Getting Started](https://plot.ly/python/getting-started/) tutorial to set up the Plotly.py library. Once that is done, let's import some libraries that we will use in this tutorial:

In [13]:
import plotly.plotly as py
import pandas as pd
import datascience
#import numpy as np

Our FIRMS dataset is quite big, so let's narrow it down to just a few points on the map. I'd like to plot the 4 brightest, 4 most "mediocre"ly bright, and 4 dimmest wildfires in the dataset. 

First, let's load the .csv file into a table:

In [14]:
data=Table().read_table('~/URAP/google_earth_fires/data/fire_archive_M6_25871.csv') #from https://github.com/jupyterhub/jupyterhub/issues/376
data

latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp
39.976,-122.757,327.8,1.2,1.1,2005-11-01,1920,Terra,MODIS,34,6.1,285.9,31.1
39.982,-122.773,329,1.3,1.1,2005-11-01,2057,Aqua,MODIS,86,6.1,278.5,43.3
39.985,-122.758,326.9,1.3,1.1,2005-11-01,2057,Aqua,MODIS,85,6.1,277.7,38.8
39.944,-122.684,305.6,1.7,1.3,2006-02-22,2141,Aqua,MODIS,51,6.1,292,16.2
39.821,-122.816,328,1.4,1.2,2006-05-03,1926,Terra,MODIS,84,6.1,296.4,38.3
39.809,-122.81,321.7,1.1,1.1,2006-05-03,2105,Aqua,MODIS,77,6.1,298.3,20.3
39.818,-122.812,351.4,1.1,1.1,2006-05-03,2105,Aqua,MODIS,96,6.1,300.1,75.6
39.814,-122.787,308.6,2.5,1.5,2006-05-04,1831,Terra,MODIS,54,6.1,292.1,24.6
40.051,-122.916,320.5,1.2,1.1,2006-06-30,625,Terra,MODIS,100,6.1,289.6,24
40.057,-122.893,311.7,1.6,1.3,2006-06-30,1037,Aqua,MODIS,83,6.1,288.5,26.2


In [15]:
Now, let's select the first three rows of this data and find the data points we want. 

SyntaxError: EOL while scanning string literal (<ipython-input-15-d1bc6ef8540b>, line 1)

In [ ]:
data=Table().read_table('~/URAP/google_earth_fires/data/fire_archive_M6_25871.csv') #from https://github.com/jupyterhub/jupyterhub/issues/376
py_data=data.select(0,1,2)
brightest=max(py_data.column('brightness'))
medium=np.median(py_data.column('brightness'))
dimmest=min(py_data.column('brightness'))
print('brightest=', brightest, 'medium=', medium, 'dimmest=', dimmest)

In [ ]:
with_bright=py_data.where('brightness', brightest)
with_medium=py_data.where('brightness', medium).take(0)
with_dim=py_data.where('brightness', dimmest).take(0)

new_data=Table().with_columns('latitude', make_array(with_bright.column(0).item(0),with_medium.column(0).item(0), with_dim.column(0).item(0)),
                             'longitude', make_array(with_bright.column(1).item(0),with_medium.column(1).item(0), with_dim.column(1).item(0)),
                              'brightness',make_array(brightest, medium, dimmest))
                              
new_data

In [ ]:
df=pd.read_csv('~/URAP/google_earth_fires/data/fire_archive_M6_25871.csv')
sorted_up=df.sort_values(by='brightness', ascending=True)

In [ ]:
a = df[(df['latitude']==38.773) & (df['brightness']==323.4)] #Baker County, Oregon
b = df[(df['latitude']==44.262) & (df['brightness']==506.2)] #Cloverdale, CA
c = df[(df['latitude']==36.082) & (df['brightness']==300)] #Tulare County, CA
three = a.append(b)
three = three.append(c)
three = three.sort_values('brightness',ascending=False)
three

In [ ]:
three.index[three['brightness']==300.0]

In [ ]:
three.iloc[0:1]

In [ ]:
three.iloc[1:2]

In [ ]:
z=three.iloc[0:1]
z['brightness']

In [ ]:
z=three.iloc[2:3]
z.at[3277, 'brightness']
#BUT HOW DO I FIND THE INDEX??

In [ ]:
len(three.index)

In [ ]:
three['text']=three['acq_date']
rows=len(three.index) #get number of rows
colors = ["rgb(255,0,0)", "rgb(255,128,0)", "rgb(255,255,0)"] #red, orange, yellow
fires=[] #the data that we want to represent on the map
for i in range(rows):
    three_sub=three.iloc[i:i+1] #grab each individual row 
    fire=dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = three_sub['longitude'],
        lat = three_sub['latitude'],
        location=['California'], #this doesn't do anything
        text = three_sub['text'],
        marker=dict(
                size=three_sub['brightness']/20,
                color=colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode='diameter'
                    ),
        name='{0}'.format(three_sub['brightness']) ) #legend labels
    fires.append(fire)

In [ ]:
layout=dict(
        title='Range of fire brightness',
        showlegend=True,
        geo = dict(
            scope='CA',
            projection=dict( type='albers usa'),
            center=dict(lon=-119.4179, lat=36.7783),
            zoom=6,
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )
fig = dict(data=fires, layout=layout)
py.iplot(fig, validate=False, filename='3-FIRMS-bubble-map')

## Resources
-  [API guide](https://plot.ly/python/user-guide/#annotations)
-  [Documentation](https://plot.ly/python/reference/)
